In [ ]:
import numpy as np
import json
import tensorflow as tf

f = open(r'../input/ships-in-satellite-imagery/shipsnet.json')
dataset = json.load(f)
f.close()

indices = [index for index, element in enumerate(dataset["labels"]) if element == 1]
imagesships = np.array([dataset['data'][i] for i in indices]).astype('uint8')
yships = np.array([dataset['labels'][i] for i in indices])
yships = np.where(yships==1, 11, yships)

color_chanels = 3
weight = 80
height = 80
Xships = imagesships.reshape([-1, color_chanels, weight, height])
Xships = np.moveaxis(Xships,1,3)

i_width = 64
i_height = 64

Xships = tf.image.resize(Xships, [i_height, i_width]).numpy()

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(10,10, figsize=(20,20))
for i,ax in enumerate(axes.flat):
    ax.imshow(Xships[i].astype('uint8'))

# **Planes Import**

In [ ]:
# download dataset from json object
ss = open(r'../input/planesnet/planesnet/planesnet.json')
dataset = json.load(ss)
ss.close()

indices = [index for index, element in enumerate(dataset["labels"]) if element == 1]
imagesplanes = np.array([dataset['data'][i] for i in indices]).astype('uint8')
yplanes = np.array([dataset['labels'][i] for i in indices])
yplanes = np.where(yplanes==1, 12, yplanes)
Xplanes = imagesplanes.reshape([-1,3, 20, 20])
Xplanes = np.moveaxis(Xplanes,1,3)

Xplanes = tf.image.resize(Xplanes, [i_height, i_width]).numpy()

In [ ]:
fig1, axes = plt.subplots(10,10, figsize=(20,20))
for i,ax in enumerate(axes.flat):
    ax.imshow(Xplanes[i].astype('uint8'))

In [ ]:
import os
from PIL import Image
from glob import glob
def read_img(location):
  data = []
  labels = []
  dirs = os.listdir(location)
  for i in dirs:
    print(location+i+'/')
    for pic in glob(location+i+'/*.jpg'):
      im = Image.open(pic)
      im = np.array(im)
      data.append(im)
      labels.append(i)
  return np.array(data),np.array(labels)

In [ ]:
X_eurosat,y_eurosat = read_img(r'../input/eurosat-dataset/EuroSAT/')

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(10,10, figsize=(15,15))
for i,ax in enumerate(axes.flat):
    ax.imshow(X_eurosat[i])

In [ ]:
Xplanes.shape

In [ ]:
Xships.shape

In [ ]:
X_eurosat.shape

In [ ]:
np.unique(y_eurosat)

In [ ]:
labels_euro=y_eurosat.copy()
labels_euro[labels_euro == "AnnualCrop"] = 1
labels_euro[labels_euro == "Forest"] = 2
labels_euro[labels_euro == "HerbaceousVegetation"] = 3
labels_euro[labels_euro == "Highway"] = 4
labels_euro[labels_euro == "Industrial"] = 5
labels_euro[labels_euro == "Pasture"] = 6
labels_euro[labels_euro == "PermanentCrop"] = 7
labels_euro[labels_euro == "Residential"] = 8
labels_euro[labels_euro == "River"] = 9
labels_euro[labels_euro == "SeaLake"] = 10

In [ ]:
labels_euro

In [ ]:
print(Xships.shape)
print(Xplanes.shape)
print(X_eurosat.shape)
print(yships.shape)
print(yplanes.shape)
print(labels_euro.shape)

In [ ]:
Xdata = np.concatenate((Xships,Xplanes,X_eurosat))
Ydata = np.concatenate((yships,yplanes,labels_euro))

In [ ]:
print(Xdata.shape)
print(Ydata.shape)

In [ ]:
Ydata

In [ ]:
labels_indices={
    1: "AnnualCrop",
    2:"Forest",
    3:"HerbaceousVegetation",
    4:"Highway",
    5:"Industrial",
    6:"Pasture",
    7:"PermanentCrop",
    8:"Residential" ,
    9:"River",
    10:"SeaLake",
    11:"Ships",
    12:"Planes"
    
}


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# plot class distributions of whole dataset
unique, counts = np.unique(Ydata, return_counts=True)
counts = dict(zip(unique, counts))



plt.figure(figsize=(10, 5))

plt.bar(range(len(counts)), list(counts.values()), align='center')
plt.xticks(range(len(counts)), list(counts.keys()), fontsize=12, rotation=80)
plt.xlabel('class label', fontsize=13)
plt.ylabel('class size', fontsize=13)
plt.title('EUROSAT+ships+planes Class Distribution', fontsize=15);

# **Construction du Modèle pour Eurosat**

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
X_train, X_test, y_train, y_test = train_test_split(X_eurosat, labels_euro, train_size=0.9, test_size=0.1, shuffle=True, random_state=42)
y_train = tf.one_hot(y_train, depth=10)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import set_image_data_format
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import optimizers, losses, utils

model = Sequential() 
model.add(Conv2D(28, (3, 3), padding="same",input_shape=(64, 64, 3)))
model.add(Activation("relu"))
model.add(Conv2D(28, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPool2D(2,2))
model.add(Conv2D(56, (3, 3),padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(56, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPool2D(2,2))
model.add(Conv2D(112, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(112, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(784))
model.add(Activation("relu"))
model.add(Dropout(0.6))
model.add(Dense(10))
model.add(Activation("sigmoid"))
adam = optimizers.Adam(lr=0.00001)
model.compile(optimizer=adam, loss=losses.binary_crossentropy, metrics=['categorical_accuracy'])

In [ ]:
history = model.fit(X_train, y_train,validation_split=0.07, epochs = 30)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model categorical_accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
y_test = tf.one_hot(y_test, depth=10)
model.evaluate(X_test,y_test)

# **Construction du Modèle pour Eurosat+Ships**

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

Xdata = np.concatenate((Xships,X_eurosat))
Ydata = np.concatenate((yships,labels_euro))

X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata, train_size=0.9, test_size=0.1, shuffle=True, random_state=42)


y_train = tf.one_hot(y_train, depth=11)



In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import set_image_data_format
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import optimizers, losses, utils


model = Sequential() 
model.add(Conv2D(28, (3, 3), padding="same",input_shape=(64, 64, 3))) 
model.add(Activation("relu")) 
model.add(Conv2D(28, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(MaxPool2D(2,2)) 
model.add(Conv2D(56, (3, 3),padding="same")) 
model.add(Activation("relu")) 
model.add(Conv2D(56, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(MaxPool2D(2,2)) 
model.add(Conv2D(112, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(Conv2D(112, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(MaxPool2D(2,2)) 
model.add(Flatten()) 
model.add(Dense(784)) 
model.add(Activation("relu")) 
model.add(Dropout(0.6)) 
model.add(Dense(11)) 
model.add(Activation("sigmoid")) 
adam = optimizers.Adam(lr=0.00001)
model.compile(optimizer=adam, loss=losses.binary_crossentropy, metrics=['categorical_accuracy'])



In [ ]:
history = model.fit(X_train, y_train,validation_split=0.07, epochs = 30)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model categorical_accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
y_test = tf.one_hot(y_test, depth=11)
model.evaluate(X_test,y_test)

# **Construction du Modèle pour Eurosat+Ships+Planes**

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

Xdata = np.concatenate((Xships,Xplanes[:3000],X_eurosat))
Ydata = np.concatenate((yships,yplanes[:3000],labels_euro))

X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata, train_size=0.9, test_size=0.1, shuffle=True, random_state=42)


y_train = tf.one_hot(y_train, depth=12)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# plot class distributions of whole dataset
unique, counts = np.unique(Ydata, return_counts=True)
counts = dict(zip(unique, counts))



plt.figure(figsize=(10, 5))

plt.bar(range(len(counts)), list(counts.values()), align='center')
plt.xticks(range(len(counts)), list(counts.keys()), fontsize=12, rotation=80)
plt.xlabel('class label', fontsize=13)
plt.ylabel('class size', fontsize=13)
plt.title('EUROSAT+ships+planes Class Distribution', fontsize=15);

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import set_image_data_format
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import optimizers, losses, utils



model = Sequential() 
model.add(Conv2D(28, (3, 3), padding="same",input_shape=(64, 64, 3))) 
model.add(Activation("relu")) 
model.add(Conv2D(28, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(MaxPool2D(2,2)) 
model.add(Conv2D(56, (3, 3),padding="same")) 
model.add(Activation("relu")) 
model.add(Conv2D(56, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(MaxPool2D(2,2)) 
model.add(Conv2D(112, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(Conv2D(112, (3, 3), padding="same")) 
model.add(Activation("relu")) 
model.add(MaxPool2D(2,2)) 
model.add(Flatten()) 
model.add(Dense(784)) 
model.add(Activation("relu")) 
model.add(Dropout(0.6)) 
model.add(Dense(12)) 
model.add(Activation("sigmoid")) 
adam = optimizers.Adam(lr=0.00001)
model.compile(optimizer=adam, loss=losses.binary_crossentropy, metrics=['categorical_accuracy'])



In [ ]:
history = model.fit(X_train, y_train,validation_split=0.07, epochs = 30)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model categorical_accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
y_test = tf.one_hot(y_test, depth=12)

model.evaluate(X_test,y_test)